In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
folder_name = ''
filename = ''
df_country = pd.read_csv(f'../data/{folder_name}/{filename}.csv')

In [3]:
df_country['Founders'].str.split(';')

0                                                    [—]
1                                                    [—]
2      [Chester Floyd Carlson,  Joseph Chamberlain Wi...
3                       [Garrett Camp,  Travis Kalanick]
4      [Adam Neumann,  Miguel McKelvey,  Rebekah Neum...
                             ...                        
995           [Jared Smith,  Ryan Smith,  Stuart Orgill]
996                                       [Craig Knight]
997                                                  [—]
998                                                  [—]
999    [Craig Roah,  Duc Chau,  Frank Addante,  Julie...
Name: Founders, Length: 1000, dtype: object

In [4]:
# results_list = []
# for i in range(1000):
#     founders_list = df_country['Founders'][i].split(';')
#     result_list = []
#     for name in founders_list:
#         name_block = name.split(' ')
#         first = ''.join(name_block[:-1])
#         last = name_block[-1]
#         result = !curl -X GET "https://v2.namsor.com/NamSorAPIv2/api2/json/gender/{first}/{last}" -H "accept: application/json" -H "X-API-KEY: d32894fde7a9f76342ee6951c4bbed7c"
#         result_list.append(result)
#     results_list.append(result_list)

In [5]:
# file = open('../data/api_results/{filename}_api.txt','w')
# file.writelines(str(results_list))
# file.close()

In [6]:
with open('../data/api_results/{filename}_api.txt', 'r') as f:
    api_result_string = f.read()

In [7]:
api_result_data = literal_eval(api_result_string)

In [8]:
literal_eval(api_result_data[2][0][5])['probabilityCalibrated']

0.902837308541162

In [9]:
df_gender = pd.DataFrame({'api_raw': api_result_data})

In [10]:
gender_data = []
prob_data = [] # will need to check through prob<0.5
female_data = [] # number of females

# loop through rows
for i in range(1000):
    
    row = df_gender.iloc[i][0]
    
    gender_list = []
    prob_list = []
    
    count = 0
        
    # loop through name list
    for name in row:
        
        # yield results
        try: 
            gender = literal_eval(name[5])['likelyGender']
            gender_list.append(gender)

            prob = literal_eval(name[5])['probabilityCalibrated']
            prob_list.append(prob)

            if (gender == 'female') and (prob > 0.5):
                count += 1
        
        # if bad request, literal_eval can't read html on name[5]
        except SyntaxError:
            gender_list.append('N/A')
            prob_list.append('N/A')
            
    # founders col was entirely empty
    if gender_list == ['N/A']:
        count = 'N/A'
    
    gender_data.append(gender_list)
    prob_data.append(prob_list)
    female_data.append(count)

In [11]:
df_gender['gender'] = gender_data
df_gender['prob'] = prob_data
df_gender['#female'] = female_data

In [12]:
# df_gender

In [13]:
# check: good to know that all probs were higher than 50%

count = 0

for prob in prob_data:
    for p in prob:
        if type(p)==float and (p < 0.5):
            count += 1

count

0

In [14]:
df = pd.merge(left=df_country, right=df_gender, left_index=True, right_index=True)

In [15]:
# df.to_csv(f'../data/{folder_name}/{filename}-gender.csv', index=False)